In [70]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

Загружаем и смотрим данные, отмечаем количество ненулевых значений в целевой колонке.

In [71]:
data_raw=pd.read_csv('..//public_train.csv')
data_raw.head(3)

,pump_id,Well_Number,Date,Volumetric_Capacity,HR,Gas_Rate,Water_Rate,Gas_Capacity,Water_Capacity,Gas_Pressure,Pump_Pressure,Pump_Speed,Pump_Torque,Pump_Volumetric_Eff,Downhole_Gauge_Fluid_Level_Above_Sensor,Pump_Torque.1,PUMP_ID_PHASE
0,NaN,AGB001,2019-05-12,NaN,NaN,0.000000,0.000000,3.114,0.016217,6.89,0.00,0.0,NaN,0.0,NaN,NaN,1.0
1,NaN,AGB001,2019-05-13,NaN,NaN,0.000000,0.006836,3.114,0.016217,1254.84,468.84,45.0,40.674600,0.0,NaN,NaN,1.0
2,NaN,AGB001,2019-05-14,NaN,NaN,2.567618,0.008850,3.114,0.016217,1500.00,664.00,59.0,55.000083,0.0,NaN,NaN,1.0


In [72]:
data_raw['HR'].notna().sum()

1779

In [73]:
print(data_raw.dtypes)
numeric__indices = np.where(data_raw.dtypes == np.float)[0]
numeric__indices

pump_id                                    float64
Well_Number                                 object
Date                                        object
Volumetric_Capacity                        float64
HR                                          object
Gas_Rate                                   float64
Water_Rate                                 float64
Gas_Capacity                               float64
Water_Capacity                             float64
Gas_Pressure                               float64
Pump_Pressure                              float64
Pump_Speed                                 float64
Pump_Torque                                float64
Pump_Volumetric_Eff                        float64
Downhole_Gauge_Fluid_Level_Above_Sensor    float64
Pump_Torque.1                              float64
PUMP_ID_PHASE                              float64
dtype: object


array([ 0,  3,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16],
      dtype=int64)

Агрегируем данные по номеру скважины и записываем в словарь где ключ номер скважины, а значение часть таблицы относящаяся к данной скважине. Переводим временную колонку в pd.datetime и устанавливаем как индекс. Заполняем пропущенные значения так же как для скважин с помошью KNN imputer, исходя из той же логики что похожие временные промежутки по каждой скважине имеют похожие значения.

In [74]:
well_dict={w:pd.DataFrame() for w in data_raw['Well_Number'].unique()}
for i in data_raw['Well_Number'].unique():
       well_dict[i]=data_raw[data_raw['Well_Number']==i]

In [76]:
well_dict_ts={wts:pd.DataFrame() for wts in well_dict.keys()}
for i in well_dict.keys():
    well_dict_ts[i]=well_dict[i].copy()
    well_dict_ts[i]['Date'] = pd.to_datetime(well_dict[i]['Date'])
    well_dict_ts[i] = well_dict[i].set_index('Date')

In [77]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=2)

In [78]:
well_dict_ts_fill={wts:pd.DataFrame() for wts in well_dict_ts.keys()}
for i in well_dict_ts_fill.keys():
    not_fill_columns=well_dict_ts[i][['pump_id', 'Well_Number', 'Volumetric_Capacity', 'HR']].copy()
    well_ts=well_dict_ts[i].copy()
    well_ts.drop(['pump_id', 'Well_Number', 'Volumetric_Capacity', 'HR'],axis=1,inplace=True)
    well_fill=imputer.fit_transform(well_ts)
    if well_ts.shape!=well_fill.shape:
        dop=len(well_ts.iloc[0])-len(well_fill[0])
        well_fill=np.column_stack((well_fill,( np.zeros([len(well_fill[:,1]),dop]))))
    well_fill_df=pd.DataFrame(well_fill,columns=well_ts.columns,index=well_ts.index)
    well_dict_ts_fill[i]=not_fill_columns.join(well_fill_df)

Функция разделяет датафрейм на временные промежутки между проверками.

In [79]:
def emergency_split(df,split_column):
    border=df.loc[df[split_column].notna()].index
    dict_spliten={k:pd.DataFrame() for k in range(len(border)+1)}
    for i in range(len(border)+1):
        if i ==0:
            dft=df[df.index<=border[i]]
            dict_spliten[i]=dft
        elif i==len(border):
            dft=df[df.index>border[i-1]]
            dict_spliten[i]=dft
        else:    
            dft=df[(df.index<=border[i])&(df.index>border[i-1])]
            dict_spliten[i]=dft
    
    return(dict_spliten)

Функция разбивает таблицу на заданное количество интервалов(независимо от длины), и усредняет значение по интервалу.

In [80]:
def meaning(df, n_space=20):
    idxs=df.index
    interval=np.linspace(0,(len(df.iloc[:,0])-1),n_space,dtype=int)
    dfr=pd.DataFrame()
    c=0
    for i in interval:
        if i ==0:
            dft=df[(df.index<=idxs[i])]
            dfr[idxs[i]]=dft.mean(axis=0)
        elif i==len(interval)-1:
            dft=df[(df.index<=idxs[i])]
            dfr[idxs[i]]=dft.mean(axis=0)
        else:
            dft=df[(df.index<=idxs[i])]
            dfr[idxs[i]]=dft.mean(axis=0)
        c=i
    return dfr.T

Функция выравнивает интервалы, либо выкидывает более ранние значения, либо добавляет в более ранние значения нули.

In [81]:
def strach_limit(df,shift=50):
    if len(df.iloc[:,1])<shift:
        l=shift-len(df.iloc[:,1])
        zes=pd.DataFrame(np.zeros([l,len(df.iloc[1])]),columns=df.columns)
        result=zes.append(df,ignore_index=True)
    else:
        result=df.tail(shift)
    return result

Создаем таблицу из временых рядов по каждой скважине, при этом мы можем регулировать глубину того на сколько мы учитываем прошлые значения, а так же на сколько интервалов мы разделим данные предшевствующие событию.

In [82]:
well_dict_preproc={wtsf:pd.DataFrame() for wtsf in well_dict_ts_fill.keys()}
for i in well_dict_preproc.keys():
    if well_dict_ts_fill[i].loc[:,'Volumetric_Capacity'].notna().sum()!=0:
        sdw=emergency_split(well_dict_ts_fill[i],'HR')
        proc=pd.DataFrame()
        for k in sdw.keys():
            if len(sdw[k].iloc[:,0])!=0:
                sw_df=strach_limit(sdw[k],35)
                series_last=sdw[k][['pump_id', 'Well_Number', 'Volumetric_Capacity', 'HR']].tail(1)
                zi=sw_df.copy()
                zi.drop(['pump_id', 'Well_Number', 'Volumetric_Capacity', 'HR'],axis=1,inplace=True)
                zi=meaning(zi,10)
                s=pd.DataFrame(zi.values.flatten()).T
                s.index=series_last.index
                d=series_last.join(s)
                proc=proc.append(d)
        well_dict_preproc[i]=proc

Смотрим результирующию таблицу для обучения. Значения NаN появились в крайних значениях временого ряда в связи с тем что целевое событие могло не случится. По этой причине мы можем спокойно удалить строки где значения пропущены.
В результате мы получили датасет равный по размеру значимым значениям в целевых колонках при этом учитывающий данные предшевствующие событи.
Сохраняем его для использования для обучения.

In [83]:
preproc_data=pd.DataFrame()
for i in well_dict_preproc.keys():
    preproc_data=preproc_data.append(well_dict_preproc[i])
preproc_data

,pump_id,Well_Number,Volumetric_Capacity,HR,0,1,2,3,4,5,...,110,111,112,113,114,115,116,117,118,119
Date,,,,,,,,,,,,,,,,,,,,,
2020-06-25,1.0,AGB001,33.0,False,77.008753,0.028822,240.816,0.056440,1500.00,528.48,...,245.264571,0.056440,1297.342857,556.598571,170.942857,95.024377,86.176333,294.590693,243.682571,1.000000
2021-01-27,NaN,AGB001,NaN,NaN,248.925848,0.021945,534.570,0.053102,1400.76,650.70,...,611.441314,0.048809,1400.086571,604.924571,83.314286,266.944620,90.072711,162.278559,256.276571,1.000000
2020-07-28,3.0,AGB002,40.0,False,0.000618,0.000000,309.324,0.037680,161.00,134.89,...,277.768800,0.038062,127.857143,104.619143,0.000000,0.000000,0.000000,391.742049,356.930000,3.000000
2021-01-27,NaN,AGB002,NaN,NaN,331.997547,0.014924,434.922,0.035136,1399.79,456.22,...,512.594057,0.032070,1329.202857,463.181143,58.971429,338.878272,63.942970,100.009299,350.908286,3.000000
2020-07-27,5.0,AGB003,33.0,False,57.609645,0.000018,253.272,0.034818,3715.00,229.13,...,258.076457,0.034818,2514.857143,197.710857,0.000000,-0.841673,0.000000,185.782150,267.779714,5.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-10-30,3746.0,ZUP208,8.0,False,0.623520,0.013065,1241.448,0.043245,272.00,267.21,...,1235.220000,0.042836,280.514286,46.540000,26.114286,29.028615,5.588059,200.115023,162.496143,3746.000000
2020-07-04,3747.0,ZUP208,13.0,True,1.765399,0.000236,402.744,0.037044,252.92,246.06,...,421.250057,0.035763,202.465714,211.198286,0.000000,0.000000,0.000000,340.398151,162.605000,3747.000000
2021-01-27,NaN,ZUP208,NaN,NaN,285.909772,0.005312,433.884,0.034818,253.73,261.25,...,448.297371,0.033714,253.263143,260.167429,64.942857,167.108399,64.743440,21.653063,165.527143,3746.042857


In [84]:
preproc_data.dropna()

,pump_id,Well_Number,Volumetric_Capacity,HR,0,1,2,3,4,5,...,110,111,112,113,114,115,116,117,118,119
Date,,,,,,,,,,,,,,,,,,,,,
2020-06-25,1.0,AGB001,33.0,False,77.008753,0.028822,240.816,0.056440,1500.00,528.48,...,245.264571,0.056440,1297.342857,556.598571,170.942857,95.024377,86.176333,294.590693,243.682571,1.0
2020-07-28,3.0,AGB002,40.0,False,0.000618,0.000000,309.324,0.037680,161.00,134.89,...,277.768800,0.038062,127.857143,104.619143,0.000000,0.000000,0.000000,391.742049,356.930000,3.0
2020-07-27,5.0,AGB003,33.0,False,57.609645,0.000018,253.272,0.034818,3715.00,229.13,...,258.076457,0.034818,2514.857143,197.710857,0.000000,-0.841673,0.000000,185.782150,267.779714,5.0
2020-06-14,7.0,AGB004,40.0,False,0.000000,0.000011,257.424,0.036567,327.00,-0.22,...,257.839200,0.036567,257.257143,-0.122571,0.000000,-0.375263,0.000000,639.442736,323.810000,7.0
2020-09-07,11.0,AGB006,33.0,False,0.000000,0.003367,185.802,0.031797,430.00,64.37,...,179.781600,0.031797,419.285714,51.680857,0.000000,-2.892899,0.000000,634.750110,207.710000,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-06-29,3740.0,ZUP206,33.0,False,0.000000,0.000709,821.058,0.076473,704.00,35.00,...,822.778114,0.076605,303.400000,58.657143,0.000000,-3.485720,0.000000,417.698973,287.192857,3740.0
2020-03-16,3741.0,ZUP206,33.0,False,54.934854,0.000055,1034.886,0.073452,900.00,4.00,...,1002.144514,0.074397,205.285714,3.200000,0.000000,-0.600001,0.000000,439.440500,292.290000,3741.0
2019-10-30,3746.0,ZUP208,8.0,False,0.623520,0.013065,1241.448,0.043245,272.00,267.21,...,1235.220000,0.042836,280.514286,46.540000,26.114286,29.028615,5.588059,200.115023,162.496143,3746.0


In [86]:
preproc_data.dropna().to_csv('preprocessed_train.csv',index=False)